In [1]:
import pandas as pd
import numpy as np

data_folder = "../data/"

df = pd.read_csv(f"{data_folder}months.csv")

df['month_name'] = df['month'].apply(lambda x: "month_"+str(x))
months_names   = ['Gennaio','Febbraio','Marzo','Aprile','Maggio','Giugno','Luglio','Agosto','Settembre','Ottobre','Novembre','Dicembre']
months_numbers = [1,2,3,4,5,6,7,8,9,10,11,12]
months_map     = dict(zip(months_numbers,months_names))

df['month_name'] = df['month'].map(months_map)

In [20]:
import plotly.express as px
fig = px.bar(df, x="month_name", y=["n_recipes","n_experiences"],facet_col='activity_school_year')
fig.show()

In [22]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Bar(y=[2, 3, 1]))
fig.show()

In [2]:
from am4chart import Amchart

In [5]:
am = Amchart()

column = am.LineChart()
column.set_y_label("number of activities")
column.set_x_label("month")
column.setHeight(450)
column.fromDataFrame(df, ["month","activity_school_year"], ["n_recipes","n_experiences"], 
                     sorter=['activity_school_year','month'],
                     hue='activity_school_year', hue_prefix="year ",
                     range_divider='activity_school_year', x_axis_names = "month",
                     visible_labels = ["10"],
                     show_legend = False, 
                    )
column.is_category()
column.plot()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#7

am = Amchart()

column = am.ColumnChart()
column.setTitle("Activity total length per month")
column.fromDataFrame(df, "month", 
                     ['avg_len_descriptions','avg_len_steps','avg_len_observations'], 
                     hue="activity_school_year", hue_prefix="year ")
column.plot()

In [ ]:
#10
am = Amchart()

column = am.ColumnChart()
column.setTitle("Activity total length per month")
column.fromDataFrame(df, "month", 
                     ['avg_len_descriptions','avg_len_steps'], 
                     hue="activity_school_year", hue_prefix="year ")
column.plot()

In [ ]:
#5

am = Amchart()

column = am.ColumnChart()
column.setTitle("Total length of average reflections per month")
column.fromDataFrame(df, "month", 
                     ['avg_len_bilancio','avg_len_competenze','avg_len_miglioramenti','avg_len_critici'], 
                     hue="activity_school_year", hue_prefix="year ")
column.plot()

In [ ]:
def prova(a,b,**args):
    print(a)
    print(b)
    print(args)
    print(args.get('cissdao','nessun_valora'))
    
prova(a='a',b='b',ciao='provazza',stocazzo='stocazzo',staminchia=0)

In [ ]:
a = {}
a['rova'] = 'azz'
if a.get('rovs'):
    print('yep')
else:
    print('note')

In [ ]:
columnData = [{
        "x": "Lithuania",
        "y": 501.9
    }, {
        "x": "Czech Republic",
        "y": 301.9
    }, {
        "x": "Ireland",
        "y": 201.1
    }, {
        "x": "Germany",
        "y": 165.8
    }, {
        "x": "Australia",
        "y": 139.9
    }, {
        "x": "Austria",
        "y": 128.3
    }, {
        "x": "UK",
        "y": 99
    }, {
        "x": "Belgium",
        "y": 800
    }, {
        "x": "The Netherlands",
        "y": 50
    }]

In [ ]:
pie = am.PieSeries()
pie.setData(df.month_name, df.n_logins)
pie.plot()

In [ ]:
minchia = False
ciao = 1 if minchia else 0
print(ciao)

In [ ]:
df[['month','n_recipes']].astype(str).set_index('month').to_dict()['n_recipes']

In [ ]:
getattr(df.groupby('A', axis=0, as_index=False), 'sum')()